# FunctionAgent with Memories


In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

## Creating a FunctionAgent with memory


In [2]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.memory import Memory
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Context

llm = OpenAI(model="gpt-4.1-mini", temperature=0)

memory = Memory.from_defaults(
    session_id="default_session", 
    token_limit=40000
)

def get_weather(location: str) -> str:
    """Usfeful for getting the weather for a given location."""
    return f"The weather at {location} is very nice with not much rain."

weather_tool = FunctionTool.from_defaults(
    get_weather,
)

agent = FunctionAgent(llm=llm, tools=[weather_tool])
ctx = Context(agent)

In [3]:
response = await agent.run("How is the weather in Tokyo?", memory=memory)
print(response)

The weather in Tokyo is very nice with not much rain.


In [4]:
response = await agent.run("How are you?  What is your name?", memory=memory)
print(response)

I'm doing well, thank you! I'm ChatGPT, your AI assistant. How can I help you today?


## Short-Term Memory


In [24]:
memory = Memory.from_defaults(
    session_id="my_session",
    token_limit=40000,
    chat_history_token_ratio=0.7,
    token_flush_size=3000
)

## Long-Term Memory


In [6]:
from llama_index.core.memory import (
    StaticMemoryBlock,
    FactExtractionMemoryBlock,
    VectorMemoryBlock,
)

In [26]:
static_memory_block = StaticMemoryBlock(
        name="core_info",
        static_content="My name is MemAgent, and I live in San Francisco.  I am here to help ASDRP researhers",
        priority=0,
    )

In [34]:
fact_extraction_memory_block = FactExtractionMemoryBlock(
        name="extracted_info",
        llm=llm,
        max_facts=50,
        priority=1,
    )

In [39]:
# First, start Qdrant locally using Docker (run this in terminal):
# docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant

from qdrant_client import AsyncQdrantClient, QdrantClient
from qdrant_client.http import models
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-small")

def setup_qdrant_with_collection(collection_name="vector_memory", vector_size=1536):
    """Setup Qdrant client and create collection if it doesn't exist"""
    try:
        q_client = QdrantClient(
            url="http://localhost:6333",
        )
        q_aclient = AsyncQdrantClient(
            url="http://localhost:6333",
        )
        
        # Check if collection exists, if not create it
        try:
            collection_info = q_client.get_collection(collection_name)
            print(f"Collection '{collection_name}' already exists")
        except Exception: # Collection doesn't exist, create it
            q_client.create_collection(
                collection_name=collection_name,
                vectors_config=models.VectorParams(
                    size=vector_size,  # This should match your embedding dimension
                    distance=models.Distance.COSINE
                )
            )
            print(f"Created collection '{collection_name}'")
        
        # Create the vector store
        vector_store = QdrantVectorStore(
            collection_name=collection_name,
            client=q_client,
            aclient=q_aclient,
            prefer_grpc=False,  # Use HTTP instead of gRPC for simplicity
        )
        
        return vector_store, q_client
        
    except Exception as e:
        print(f"Error connecting to Qdrant: {e}")
        return None, None

# Setup Qdrant
vector_store, qdrant_client = setup_qdrant_with_collection()

if vector_store is not None:
    vector_memory_block = VectorMemoryBlock(
        name="vector_memory",
        vector_store=vector_store,
        embed_model=embed_model,
        priority=2,
    )
    print("VectorMemoryBlock created successfully with Qdrant!")
else:
    print("Skipping VectorMemoryBlock creation due to Qdrant connection issues")
    vector_memory_block = None

Collection 'vector_memory' already exists
VectorMemoryBlock created successfully with Qdrant!


In [40]:
long_term_memory_blocks = [
    static_memory_block,
    fact_extraction_memory_block,
    vector_memory_block,
]

In [41]:
from llama_index.core.memory import Memory, InsertMethod

memory = Memory.from_defaults(
    session_id="my_session",
    token_limit=40000,
    chat_history_token_ratio=0.7,
    token_flush_size=3000,
    memory_blocks=long_term_memory_blocks,
    insert_method=InsertMethod.SYSTEM
)

print(memory.get_all())

[]


In [42]:
response = await agent.run("How is the weather in Tokyo?", memory=memory)
print(response)

print(memory.get_all())

The weather in Tokyo is very nice with not much rain.
[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='How is the weather in Tokyo?')]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_tJoUd4foVLAU17KoayZOY1Y3', 'function': {'arguments': '{"location":"Tokyo"}', 'name': 'get_weather'}, 'type': 'function'}]}, blocks=[TextBlock(block_type='text', text='')]), ChatMessage(role=<MessageRole.TOOL: 'tool'>, additional_kwargs={'tool_call_id': 'call_tJoUd4foVLAU17KoayZOY1Y3'}, blocks=[TextBlock(block_type='text', text='The weather at Tokyo is very nice with not much rain.')]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The weather in Tokyo is very nice with not much rain.')])]


In [46]:
messages = memory.get_all()
len(messages)

4

In [59]:
from pprint import pprint
for message in memory.get_all():
    print(f"=> role: {message.role}")
    if message.content:
        print(f"\tCONTENT: {message.content}")
    if 'tool_calls' in message.additional_kwargs.keys():
        for tool_call in message.additional_kwargs['tool_calls']:
            print(f"\tTOOL: {tool_call['function']['name']}: {tool_call['function']['arguments']}")

=> role: MessageRole.USER
	CONTENT: How is the weather in Tokyo?
=> role: MessageRole.ASSISTANT
	TOOL: get_weather: {"location":"Tokyo"}
=> role: MessageRole.TOOL
	CONTENT: The weather at Tokyo is very nice with not much rain.
=> role: MessageRole.ASSISTANT
	CONTENT: The weather in Tokyo is very nice with not much rain.
